In [13]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [52]:
files = glob('subtopics/*.json')
len(files)

12503

In [53]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}page-{i}')

100%|███████████████████████████████████| 12503/12503 [00:12<00:00, 1017.16it/s]


In [56]:
urls = sorted(urls)

In [62]:
len(urls), len(urls) // 50

(1621140, 32422)

In [60]:
import time

def get_href_topic(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find_all('div', {'class': 'bbWrapper'})
    return [str(d) for d in div]


filtered = get_href_topic(urls[0])

In [63]:
!mkdir pages

In [64]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href_topic, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

 40%|████████████▍                  | 12990/32423 [15:28:42<23:29:00,  4.35s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 98%|█████████████████████████████████▏| 31632/32423 [38:44:18<55:54,  4.24s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
!du -hs pages

15G	pages


In [67]:
# !zip -r pages.zip pages

In [68]:
files = glob('subtopics/*.json')
len(files)

12503

In [75]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d[1]:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|█████████████████████████████████████| 12503/12503 [08:43<00:00, 23.90it/s]


In [76]:
len(texts)

2458406

In [77]:
files = glob('pages/*.json')
len(files)

32423

In [83]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        for d_ in d:
            texts.append(BeautifulSoup(d_, "lxml").text)

100%|███████████████████████████████████| 32423/32423 [1:03:55<00:00,  8.45it/s]


In [84]:
len(texts)

17795846

In [85]:
texts = list(set(texts))
len(texts)

15502920

In [95]:
with open('everything.jsonl', 'w') as fopen:
    for t in tqdm(texts):
        if len(t):
            fopen.write(f'{json.dumps(t)}\n')

100%|███████████████████████████| 15502920/15502920 [00:25<00:00, 619735.63it/s]


In [96]:
!head -n 10 everything.jsonl

"sekarang sesuai kot nak bukak hedge.. tutup semua sell EG (tutup sell eu buy gu) dan biar equity naik.. apa pendapat masta2 semua?"
"06.07.20 13:32\tAccount\tReceive\t+4.39\tReceived Payment 4.39 USD from account U13301712 to account U20. Batch: 318188862. Memo: \u041e\u043f\u043b\u0430\u0442\u0430 bounty-\u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u044b \u043e\u0442 \u043f\u043e\u0440\u0442\u0430\u043b\u0430 Profit-Hunters.biz"
"tau lak mencicit2 lari.)"
"\n\ninsyi\u00a9 said:\n\n\n\npiii....raaa...kuar je pintu lift, muka dia ni ngan mael la aku nmpk dulu...posing sakan ngan gadis2 berbaju kebaya tuh...)\n\nsiap dpt hamper lg tu...:-w :-w\n\nClick to expand...\n\n\n\nsmlm dtg anta mael je.dia tak tahu jln..pastu trus balik.."
"aku kije la twinzz..\n\nko tu tawu mntak je kt mak..\n\nxbagi je majowk )"
"\n\nsuffian13 said:\n\n\n\n\nbro...\nput me in ur list\ndone to clik\nIP Anda :\n115.134.210.31\n\nhttp://www.dapatwang.com/?id=suffian13\n\n(list ni aku main klik je tanpa berkir

In [97]:
!wc -l everything.jsonl

15502919 everything.jsonl
